# DETAILED LONGITUDINAL TEST NOTEBOOK 

```
LAST UPDATE: 17-01-2024
SCRIPTs: /data/projects/service_sector_update/multiprocessing/..
```

This notebnook contains the code to be run in order to check if the output split files are correct <br>
NB: some of the splits are very big, take care of RAM usage

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc 

In [3]:
DL_SPLITS_PATH = '/data/processed_data/detailed_longitudinal/R10/service_sector/splits/' 

In [4]:
OUTPUT_COLUMNS = {
    "FINREGISTRYID":str,
    "SOURCE":str,
    "EVENT_AGE":float,
    "EVENT_DAY":str,
    "CODE1":str,
    "CODE2":str, 
    "CODE3":str, 
    "CODE4":str, 
    "CODE5":str,
    "CODE6":str, 
    "CODE7":str,
    "CODE8":str,
    "CODE9":str,
    "ICDVER":str,
    "CATEGORY":str,
    "INDEX":int}

# TEST FUNCTIONS

In [5]:
def check_inpat_freq(df):
    df.EVENT_DAY =  pd.to_datetime(df.EVENT_DAY, format='%Y-%m-%d')
    df['YEAR'] = df.EVENT_DAY.dt.year
    source_counts_by_year = df.groupby(['YEAR', 'SOURCE']).size().unstack(fill_value=0)
    source_counts_by_year.plot(kind='bar', stacked=True)
    plt.xlabel('YEAR')
    plt.ylabel('Frequency')
    plt.legend(title='SOURCE', bbox_to_anchor=(1, 1), loc='upper left')
    plt.show()

In [6]:
def check_source(df):
    out = df.SOURCE.unique()
    print(out)
    # assert only if not hilmo or avohilmo
    if not set(out).issubset({'INPAT', 'OUTPAT', 'OPER_IN', 'OPER_OUT', 'PRIM_OUT'}): 
        assert len(out)==1 ,"more than one value, wrong!"
        print('only one source value, correct!')
    else:
        print('check source value')

In [7]:
def check_event_age(df):
    m = df.EVENT_AGE.min()
    print(m)
    assert m>0 ,"min value is wrong!"
    M = df.EVENT_AGE.max()
    print(M)
    assert M<=110 ,"max value is wrong!"
    print('age range is correct!')
    
    # plot distribution
    plt.hist(df.EVENT_AGE, bins=20)
    plt.xlabel('AGE')
    plt.ylabel('Frequency')
    plt.show()

In [8]:
def check_event_day(df):
    df.EVENT_DAY =  pd.to_datetime(df.EVENT_DAY, format='%Y-%m-%d')
    year = df.EVENT_DAY.dt.year
    m = year.min()
    print(m)
    if m<1953: print("min value is wrong!")
    M = year.max()
    print(M)
    if M>2021: print("max value is wrong!")
    print('this is the registry year range: '+str(m)+'-'+str(M))
    
    # plot distribution
    plt.hist(year, bins=20)
    plt.xlabel('EVENT YEAR')
    plt.ylabel('Frequency')
    plt.show()

In [9]:
def check_icdver(df):
    out = df.ICDVER.unique()
    print(out)
    assert set(out).issubset({'8','9','10','O3'}) ,"icd version has a wrong value!"
    print('icd version values are correct!')

In [10]:
def check_category(df):
    out = df.CATEGORY.unique()
    print('number of distict category values: '+str(len(out)))
    if len(out)<=5: print(out)

In [11]:
def check_min_subjects(values, df, col, id_col):
    res = np.array([], dtype=str)
    values = np.array(values)
    if col == id_col:
        res = np.append(res, MIN_SUBJECTS_STR)
    elif id_col:
        for value in values[~pd.isnull(values)]:
            n_subjects = len(set(df.loc[df[col] == value, id_col].values))
            value = value if n_subjects >= MIN_SUBJECTS else MIN_SUBJECTS_STR
            res = np.append(res, value)
            if value == MIN_SUBJECTS_STR:
                break
    else:
        res = values
    res = " ".join(res.astype(str))
    return res

In [12]:
def run_baseline_tests(df):
    check_source(df)
    print('---')
    check_event_age(df)
    print('---')
    check_event_day(df)
    print('---')
    check_icdver(df)
    print('---')
    check_category(df)
    print('---')

In [13]:
# copying summary stat function from here:
# https://github.com/dsgelab/finregistry-data/blob/main/finregistry_data/profiling.py

MIN_SUBJECTS = 5
MIN_SUBJECTS_STR = "<not enough subjects>"

def run_summary_stats(df, id_col=None):
    nrows, ncols = df.shape

    d = {
        "col": df.columns,
        "missing_pct": [],
        "unique_values": [],
        "min": [],
        "max": [],
        "top5": [],
    }

    for col in df:

        counts = df[col].value_counts()
        missing_pct = round(df[col].isin(['nan',np.NaN]).sum(axis=0) / nrows * 100, 2)
        unique_values = len(counts)
        min_value = check_min_subjects([counts.index.min()], df, col, id_col)
        max_value = check_min_subjects([counts.index.max()], df, col, id_col)
        top5_values = check_min_subjects(counts[counts >= MIN_SUBJECTS].head(5).index, df, col, id_col)

        d["missing_pct"].append(missing_pct)
        d["unique_values"].append(unique_values)
        d["min"].append(min_value)
        d["max"].append(max_value)
        d["top5"].append(top5_values)

    profiles = pd.DataFrame.from_dict(d)

    print(f"Number of rows: {nrows}")
    print(f"Number of columns: {ncols}")
    print("Column profiles:")
    print(profiles)

# HILMO

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Hilmo_1969_1986.csv')
df = df.astype(OUTPUT_COLUMNS) 
check_inpat_freq(df)
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Hilmo_1987_1993.csv')
df = df.astype(OUTPUT_COLUMNS) 
check_inpat_freq(df)
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Hilmo_1994_1995.csv')
df = df.astype(OUTPUT_COLUMNS) 
check_inpat_freq(df)
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
# too big for memory load
df = pd.read_csv(DL_SPLITS_PATH+'Hilmo_1996_2018.csv', nrows=10_000_000)
df = df.astype(OUTPUT_COLUMNS) 
check_inpat_freq(df)
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Hilmo_2019_2021.csv')
df = df.astype(OUTPUT_COLUMNS) 
check_inpat_freq(df)
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

# AVOHILMO

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_icd10_11_16.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_icd10_17_19.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_icd10_20_21.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_icpc2_11_16.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_icpc2_17_19.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_icpc2_20_21.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_oper_11_16.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_oper_17_19.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_oper_20_21.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_oral_11_16.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_oral_17_19.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Avohilmo_oral_20_21.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

# CANCER

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Cancer.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

# DEATH

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'Death.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

# KELA REIMBURSEMENT

In [ ]:
df = pd.read_csv(DL_SPLITS_PATH+'KelaReimbursement.csv')
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()

# KELA PURCHASES

In [ ]:
# too big for memory load
df = pd.read_csv(DL_SPLITS_PATH+'KelaPurchase.csv', nrows=10_000_000)
df = df.astype(OUTPUT_COLUMNS) 
run_baseline_tests(df)
run_summary_stats(df)
del df
gc.collect()